In [5]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.keys import Keys


from selenium import webdriver
from selenium.webdriver.common.keys import Keys


## Scrapper de Carrefour

In [26]:


def scrap_function(url, num_products_to_scrape, name_class_product_grid_item, name_class_product_card_title_text, name_product_link, name_image, name_origin, name_price, name_price_per_unit, name_weight,bouton_cookies = True):
    # Initialiser le navigateur Chrome
    #chrome_path = "/Users/victor/dev/chromedriver/ChromeDriver 114.0.5735.90/chromedriver" 
    #chrome_service = ChromeService(chrome_path)
    #driver = webdriver.Chrome()


    options = webdriver.SafariOptions()
    driver = webdriver.Safari(options=options)

    # Accéder à l'URL
    driver.get(url)

    time.sleep(5)

    # scraped_product_names stock les produits déja récup par le scrap
    scraped_product_names = set()
    if bouton_cookies:
    # scipt poue le bouton des cookies
        driver.execute_script("""
            var rejectButton = document.getElementById("onetrust-reject-all-handler");
            if (rejectButton) {
                rejectButton.click();
            } else {
                console.log("Le bouton n'a pas été trouvé.");
            }
        """)

    # liste pour stocker les données
    data = []



    i = 0
    # Boucle pour scraper le nombre spécifié de produits
    while len(data) < num_products_to_scrape:
        # Récupérer les éléments de la classe "product-grid-item"
        product_cards = driver.find_elements(By.CLASS_NAME, name_class_product_grid_item)

        # Parcourir les cartes de produits
        for product_card in product_cards:
            try:
                # Récupérer le nom du produit
                name = product_card.find_element(By.CLASS_NAME, name_class_product_card_title_text).text if product_card.find_elements(By.CLASS_NAME, name_class_product_card_title_text) else None

                # Vérifier si le produit a déjà été récupéré
                if name not in scraped_product_names:
                    #recuperer le lien du produit
                    product_link = product_card.find_element(By.CLASS_NAME, name_product_link).get_attribute("href") if product_card.find_elements(By.CLASS_NAME, name_product_link) else None

                    # Récupérer le lien de l'image
                    #image_link = product_card.find_element(By.CLASS_NAME, "image image--hidden product-card-image__image ls-is-cached image--loaded").get_attribute("src") if product_card.find_elements(By.CLASS_NAME, "main-horizontal__image") else None
                    image_element = driver.find_element(By.CLASS_NAME, name_image)
                    image_link = image_element.get_attribute('src')
                    # Récupérer l'origine du produit
                    origin = product_card.find_element(By.CLASS_NAME, name_origin).text if product_card.find_elements(By.CLASS_NAME, name_origin) else None

                    # Récupérer le prix
                    price = product_card.find_element(By.CLASS_NAME, name_price).text if product_card.find_elements(By.CLASS_NAME, name_price) else None

                    # Récupérer le prix à l'unité
                    price_per_unit = product_card.find_element(By.CLASS_NAME, name_price_per_unit).text if product_card.find_elements(By.CLASS_NAME, name_price_per_unit) else None

                    #recuperer le poids
                    weight = product_card.find_element(By.CLASS_NAME, name_weight).text if product_card.find_elements(By.CLASS_NAME, name_weight) else None


                    # Ajouter les données à la liste
                    data.append({
                        "lien du produit": product_link,
                        'Nom': name,
                        'Image': image_link,
                        'Origine': origin,
                        'Prix': price,
                        'Prix à l\'unité': price_per_unit,
                        'Poids': weight,
                    })

                    # Ajouter le nom du produit à l'ensemble des produits déjà récupérés
                    scraped_product_names.add(str(name)+'_'+str(i))
                    i += 1
            except Exception as e:
                print(f"Erreur lors du traitement d'une carte de produit : {e}")
        # Ajouter le scroll en bas de la page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 1500);")

        time.sleep(1)
        try:
            # Vérifier si le bouton "Voir plus" est présent
            see_more_button = driver.find_element(By.CLASS_NAME, "pagination__button-wrap").find_element(By.TAG_NAME, "button")
            print("Il y a un bouton")
            # Cliquer sur le bouton "Voir plus"
            see_more_button.click()
            # Attendre un court instant pour le chargement
            time.sleep(2)
        except:
            print("Il n'y a plus de bouton")
            # Quitter la boucle s'il n'y a plus de bouton "Voir plus"
            break

    # Fermer le navigateur
    driver.quit()

    # on créé un DataFrame à partir des données
    df = pd.DataFrame(data)

    return df




In [ ]:
#POUR CARREFOUR

url = "https://www.carrefour.fr/r/fruits-et-legumes/legumes"
num_products_to_scrape = 50

# grille de produits
name_class_product_grid_item = "product-grid-item"
#nom du produit
name_class_product_card_title_text = "product-card-title__text"
#lien du produit
name_product_link = "product-card-image"
#image du produit
name_image = "product-card-image__image"
#origine du produit
name_origin = "ds-product-card-refonte__origin-text"    
#prix du produit
name_price = "product-price__amount-value"
#prix à l'unité du produit
name_price_per_unit = "ds-product-card-refonte__perunitlabel"
#poids du produit
name_weight = "product-card-title__weight"

df = scrap_function(url, num_products_to_scrape, name_class_product_grid_item, name_class_product_card_title_text, name_product_link, name_image, name_origin, name_price, name_price_per_unit, name_weight)

df

In [22]:
df

,lien du produit,Nom,Image,Origine,Prix,Prix à l'unité,Poids
0,https://www.carrefour.fr/p/concombre-300000103...,Concombre,https://media.carrefour.fr/medias/c0ea97f78e12...,FRANCE,"1,19",1.19 € / U,None
1,https://www.carrefour.fr/p/radis-roses-3276557...,Radis roses,https://media.carrefour.fr/medias/c0ea97f78e12...,FRANCE,"1,49",1.49 € / U,None
2,https://www.carrefour.fr/p/avocat-bio-carrefou...,Avocat Bio CARREFOUR BIO,https://media.carrefour.fr/medias/c0ea97f78e12...,ESPAGNE,"1,00",1.00 € / U,None
3,https://www.carrefour.fr/p/pommes-de-terre-de-...,Pommes de terre de consommation Vapeur CARREF...,https://media.carrefour.fr/medias/c0ea97f78e12...,FRANCE,"2,49",1.66 € / KG,None
4,https://www.carrefour.fr/p/courge-decouverte-b...,Courge découverte Bio,https://media.carrefour.fr/medias/c0ea97f78e12...,FRANCE,"2,19",2.19 € / U,None
...,...,...,...,...,...,...,...
89,https://www.carrefour.fr/p/haricot-vert-bio-32...,Haricot vert bio,https://media.carrefour.fr/medias/c0ea97f78e12...,FRANCE,"4,99",9.98 € / KG,None
90,https://www.carrefour.fr/p/haricot-vert-352368...,Haricot vert,https://media.carrefour.fr/medias/c0ea97f78e12...,ORIGINE C.E.E.,"2,50",5.56 € / KG,None
91,https://www.carrefour.fr/p/jeunes-pousses-d-ep...,Jeunes pousses d'épinard CARREFOUR,https://media.carrefour.fr/medias/c0ea97f78e12...,FRANCE,"1,40",14.00 € / KG,None
92,None,None,https://media.carrefour.fr/medias/c0ea97f78e12...,None,None,None,None


## Ce code Fonctionne bien pour scraper les infos des légumes carrefour

In [ ]:
# Enregistrer le DataFrame au format CSV si nécessaire
df.to_csv('fruits_data_carrefour.csv', index=False)

## Ici on teste pour d'autre plateforme de courses en ligne : Intermaché et Chronodrive

In [ ]:
#POUR Auchan

url = "https://www.intermarche.com/rayons/fruits-et-legumes/legumes/7570"
num_products_to_scrape = 50

# grille de produits
name_class_product_grid_item = "stime-product-list__grid"
#nom du produit
name_class_product_card_title_text = "infoTile__title"
#lien du produit
name_product_link = "link link--link product__info undefined isTileLot isTileLot--CATEGORY"
#image du produit
name_image = "image product__image isTileLot no-flag"
#origine du produit
name_origin = "infoTile__origin"    
#prix du produit
name_price = "product-tile__crossedOutPriceYellow"
#prix à l'unité du produit
name_price_per_unit = "product-tile__unit-price"
#poids du produit
name_weight = "infoTile__packaging"

df2= scrap_function(url, num_products_to_scrape, name_class_product_grid_item, name_class_product_card_title_text, name_product_link, name_image, name_origin, name_price, name_price_per_unit, name_weight,bouton_cookies = False)

In [ ]:
#Chronodrive


url = "https://www.chronodrive.com/fruits--legumes/legumes-R16482636"
num_products_to_scrape = 50

# grille de produits
name_class_product_grid_item = "product-grid-line"
#nom du produit
name_class_product_card_title_text = "text-heading card-label"
#lien du produit
name_product_link = "link link--link product__info undefined isTileLot isTileLot--CATEGORY"
#image du produit
name_image = "media"
#origine du produit
name_origin = "infoTile__origin"    
#prix du produit
name_price = "product-tile__crossedOutPriceYellow"
#prix à l'unité du produit
name_price_per_unit = "product-tile__unit-price"
#poids du produit
name_weight = "infoTile__packaging"

def scrap_function(url, num_products_to_scrape, name_class_product_grid_item, name_class_product_card_title_text, name_product_link, name_image, name_origin, name_price, name_price_per_unit, name_weight,bouton_cookies = True):
    # Initialiser le navigateur Chrome
    #chrome_path = "/Users/victor/dev/chromedriver/ChromeDriver 114.0.5735.90/chromedriver" 
    #chrome_service = ChromeService(chrome_path)
    #driver = webdriver.Chrome()


    options = webdriver.SafariOptions()
    driver = webdriver.Safari(options=options)

    # Accéder à l'URL
    driver.get(url)

    time.sleep(12)

    # scraped_product_names stock les produits déja récup par le scrap
    scraped_product_names = set()
    if bouton_cookies:
    # scipt poue le bouton des cookies
        rejectButton = driver.find_element(By.CLASS_NAME, "didomi-continue-without-agreeing")
        rejectButton.click()

        #driver.execute_script("""
        #    var rejectButton = document.getElementByClassName("didomi-continue-without-agreeing");
        #    if (rejectButton) {
        #        rejectButton.click();
        #    } else {
        #        console.log("Le bouton des cookies n'a pas été trouvé.");
        #    
        #""")
    time.sleep(4)
    if map:
        #trouver le bouton grace au Xpath
        bouton_map = driver.find_element(By.XPATH,'/html/body/div[3]/div[2]/div/div/div/div/div[1]/div[2]/div/div/div[1]/div/div[3]/div[1]/div[2]/div/div[3]/div[33]')
        #cliquer sur le bouton
        bouton_map.click()

    time.sleep(2)
    driver.execute_script("""
        var chosirbutton = document.getElementByClass('ui-cta locator-item-pick -layout-solid -size-m -variant-1');
        if (chosirbutton) {
            chosirbutton.click();
        } else {
            console.log("Le bouton n'a pas été trouvé.");
        }
    """)

    
    # liste pour stocker les données
    data = []



    i = 0
    # Boucle pour scraper le nombre spécifié de produits
    while len(data) < num_products_to_scrape:
        # Récupérer les éléments de la classe "product-grid-item"
        product_cards = driver.find_elements(By.CLASS_NAME, name_class_product_grid_item)

        # Parcourir les cartes de produits
        for product_card in product_cards:
            try:
                # Récupérer le nom du produit
                name = product_card.find_element(By.CLASS_NAME, name_class_product_card_title_text).find_element(By.TAG_NAME, 'b') if product_card.find_elements(By.CLASS_NAME, name_class_product_card_title_text).find_element(By.TAG_NAME, 'b') else None
                print(name)
                # Vérifier si le produit a déjà été récupéré
                if name not in scraped_product_names:
                    #recuperer le lien du produit
                    product_link = product_card.find_element(By.CLASS_NAME, name_product_link).get_attribute("href") if product_card.find_elements(By.CLASS_NAME, name_product_link) else None

                    # Récupérer le lien de l'image
                    #image_link = product_card.find_element(By.CLASS_NAME, "image image--hidden product-card-image__image ls-is-cached image--loaded").get_attribute("src") if product_card.find_elements(By.CLASS_NAME, "main-horizontal__image") else None
                    image_element = driver.find_element(By.CLASS_NAME, name_image)
                    image_link = image_element.get_attribute('src')
                    # Récupérer l'origine du produit
                    origin = product_card.find_element(By.CLASS_NAME, name_origin).text if product_card.find_elements(By.CLASS_NAME, name_origin) else None

                    # Récupérer le prix
                    price = product_card.find_element(By.CLASS_NAME, name_price).text if product_card.find_elements(By.CLASS_NAME, name_price) else None

                    # Récupérer le prix à l'unité
                    price_per_unit = product_card.find_element(By.CLASS_NAME, name_price_per_unit).text if product_card.find_elements(By.CLASS_NAME, name_price_per_unit) else None

                    #recuperer le poids
                    weight = product_card.find_element(By.CLASS_NAME, name_weight).text if product_card.find_elements(By.CLASS_NAME, name_weight) else None


                    # Ajouter les données à la liste
                    data.append({
                        "lien du produit": product_link,
                        'Nom': name,
                        'Image': image_link,
                        'Origine': origin,
                        'Prix': price,
                        'Prix à l\'unité': price_per_unit,
                        'Poids': weight,
                    })

                    # Ajouter le nom du produit à l'ensemble des produits déjà récupérés
                    scraped_product_names.add(str(name)+'_'+str(i))
                    i += 1
            except Exception as e:
                print(f"Erreur lors du traitement d'une carte de produit : {e}")
        # Ajouter le scroll en bas de la page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 1500);")

        time.sleep(1)
        try:
            # Vérifier si le bouton "Voir plus" est présent
            see_more_button = driver.find_element(By.CLASS_NAME, "pagination__button-wrap").find_element(By.TAG_NAME, "button")
            print("Il y a un bouton")
            # Cliquer sur le bouton "Voir plus"
            see_more_button.click()
            # Attendre un court instant pour le chargement
            time.sleep(2)
        except:
            print("Il n'y a plus de bouton")
            # Quitter la boucle s'il n'y a plus de bouton "Voir plus"
            break

    # Fermer le navigateur
    driver.quit()

    # on créé un DataFrame à partir des données
    df = pd.DataFrame(data)

    return df


df2= scrap_function(url, num_products_to_scrape, name_class_product_grid_item, name_class_product_card_title_text, name_product_link, name_image, name_origin, name_price, name_price_per_unit, name_weight,bouton_cookies = True)
